In [1]:
import torch
torch.__version__

'2.0.0+cu117'

In [2]:
import os
import numpy as np
import lightning as L

from box import Box

from robertuito import evaluate, predict, train_model
from robertuito import create_folds, import_data

L.seed_everything(42, workers=True)

# solving forking issues
os.environ["TOKENIZERS_PARALLELISM"] = "false"

/home/alfonso/miniconda3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 42


In [3]:
LABELS = [
    "Odio",
    # "Mujeres",
    # "Comunidad LGBTQ+",
    # "Comunidades Migrantes",
    # "Pueblos Originarios",
]

TRAIN_PATH = "public_data/tweets_train.csv"
TEST_PATH = "Datathon-2022-full/Datos/tweets_test.csv"
train, test = import_data(TRAIN_PATH, TEST_PATH)
train = create_folds(train, LABELS)

In [4]:
dm_config = Box(dict(
    train = train,
    test = test,
    labels = LABELS,
    batch_size = 32,
    tokenizer = 'pysentimiento/robertuito-hate-speech',
))

model_config = Box(dict(
    model_name = 'pysentimiento/robertuito-hate-speech',
    dropout = 0.2,
    hidden_size = 768,
    n_labels = len(LABELS),
    train_size = 56,
    batch_size = dm_config.batch_size,
    warmup=0.2,
    w_decay=0.001,
    lr = 3e-4
))

training_config = Box(dict(
    max_epochs = 1,
    patience = 10,
    fast_dev_run=False,
    overfit_batches=0
))

In [5]:
%%time
score = []
preds = np.empty((len(test), 5))
for fold in range(5):
    trainer, model, dm= train_model([fold], dm_config, model_config, training_config)
    f1 = evaluate(trainer, model, dm, threshold = 0.5, custom=False)
    preds[:, fold] = predict(trainer, model, dm, validation=False).reshape(-1)
    print(f"Fold {fold} F1: {f1}")
    score.append(f1)

Some weights of the model checkpoint at pysentimiento/robertuito-hate-speech were not used when initializing RobertaModel: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at pysentimiento/robertuito-hate-speech and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inf

Epoch 0: 100%|██████████| 56/56 [00:08<00:00,  6.27it/s, v_num=76, train_loss=0.710, val_loss=0.682]

Epoch 0, global step 56: 'val_loss' reached 0.68212 (best 0.68212), saving model to '/home/alfonso/Documents/hate_speech/checkpoints/epoch=0-step=56-v36.ckpt' as top 1
`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 56/56 [00:11<00:00,  4.74it/s, v_num=76, train_loss=0.710, val_loss=0.682]


You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

Epoch 0: 100%|██████████| 56/56 [00:12<00:00,  4.46it/s, v_num=80, train_loss=0.563, val_loss=0.500]


You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 15/15 [00:00<00:00, 18.74it/s]


You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 72/72 [00:03<00:00, 22.16it/s]
Fold 4 F1: 0.7808764940239045
CPU times: user 1min 23s, sys: 24.3 s, total: 1min 48s
Wall time: 2min 25s


In [ ]:
import pandas as pd
pd.read

In [6]:
score

[0.7444444444444445,
 0.7957894736842106,
 0.8,
 0.8484848484848485,
 0.7808764940239045]

In [7]:
print(f"Mean 5 Fold CV Score: {np.mean(score)}")

Mean 5 Fold CV Score: 0.7939190521274817


In [8]:
from sklearn.metrics import f1_score
y_pred = np.where(preds.mean(axis = 1) >= 0.5, 1, 0)
f1_score(test.Odio.values, y_pred)

0.8258011503697616